# Imports

In [1]:
%matplotlib inline

import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# jupyter notebook cannot handle parent dir imports without this
sys.path.append(os.path.dirname(os.getcwd()))

import time
import torch, torchvision
import numpy as np
import cv2
from scripts.renderer.visual_hull import VisualHullRenderer
from scripts.renderer.absorption_only import AbsorptionOnlyRenderer
from scripts.renderer.emission_absorption import EmissionAbsorptionRenderer
import scripts.renderer.transform as dt
import scripts.utils.io as io
import scripts.utils.utils as utils

import ipywidgets as widgets
from ipywidgets import Layout, Video
from IPython.display import display

from attrdict import AttrDict

config = {'device' : 'cpu', 'data' : {'cube_len': 64}}
param = AttrDict(config)
transform = dt.Transform(param)

# TODO update path
path = '../scripts/tests/input_data/object.raw'

[INFO] setup transform


# Options 

In [2]:
x = widgets.FloatSlider(value=0.0,min=-1,max=1,step=0.1, description='x')
y = widgets.FloatSlider(value=0.0,min=-1,max=1,step=0.1, description='y')
z = widgets.FloatSlider(value=-1.0,min=-1,max=1,step=0.1, description='z')
display(x,y,z)

w_renderer = widgets.Select(options=['VH', 'AO', 'EA'], value='EA',description='Renderer:')

display(w_renderer)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='y', max=1.0, min=-1.0)

FloatSlider(value=-1.0, description='z', max=1.0, min=-1.0)

Select(description='Renderer:', index=2, options=('VH', 'AO', 'EA'), value='EA')

# Load renderer

In [7]:
if w_renderer.value == 'VH':
    renderer = VisualHullRenderer(param)
elif w_renderer.value == 'AO':
    renderer = AbsorptionOnlyRenderer(param)
elif w_renderer.value == 'EA':
    renderer = EmissionAbsorptionRenderer(param)

# Load voxels 

In [8]:
volume = io.read_volume(path)
volume = utils.numpy_to_pytorch(volume).unsqueeze(0)


In [9]:
vector = torch.tensor([[x.value,y.value,z.value]])
image = renderer.render(transform.rotate_by_vector(volume, vector))


### Output

In [10]:
def get_image(array):
    array = torchvision.utils.make_grid(array)
    array = utils.convert_to_int(array)
    array = utils.pytorch_to_numpy(array, is_batch=False)
    
    array = cv2.cvtColor(array, cv2.COLOR_RGB2BGR)
    _, array = cv2.imencode('.png', array)
    array = array.tobytes()
    image = widgets.Image(value=array,format='png',width=300,height=300)
    return image


display(get_image(image))

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00@\x00\x00\x00@\x08\x02\x00\x00\x00%\x0b\xe6\x89\x…